This code is meant to help you get started with your project and it adapted from examples contained here: https://github.com/esdalmaijer/PyGaze/tree/master/examples. Please note that this code requires the eye tracker, so do not try to run the code yet unless you want to comment out the tracker lines and play with the display. 

!pip install keyboard
#!pip install PyLink
#!pip install pylink==0.3.2
#!pip install --index-url=https://pypi.sr-support.com sr-research-pylink

In [ ]:
#!pip install -r requirements.txt

In [ ]:
#import os
#path = 'C:\\Users\\Chloe\\Documents\\GitHub\\EyeGazeCapstone'
#os.chdir(path)

In [ ]:
# import packages
import os
import random
import time
import keyboard
import pylink

from pygaze.libscreen import Display, Screen
from pygaze.libinput import Keyboard
from pygaze.eyetracker import EyeTracker
from pygaze.liblog import Logfile
import pygaze.libtime as timer
from pygaze.mouse import Mouse

import numpy as np
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings('ignore')


# Experiment variables

In [ ]:
time.strftime("%y%m%d", time.localtime())

In [ ]:
# Name of participant
USER_NAME = 'Chloe'

# 1. Device positioning
#    The angle between device and eyes (e.g. lower, upper, lowerleft)
ANGLE = ''                
#    The distance between the participant and the display (cm)
SCREENDIST = 60.0             


# 2. Noise (distractions on page, quick blinks, eyes out of screen, etc.)
#    The noisy situations in experiment (e.g. dstr, qb, eo)
NOISE = ''                    


# 3. Display time: time of image displayed on the screen
#    Maximum length of time for each image to be displayed (ms)
MAXTRIALTIME = 10000         


# 4. Fixation time: time of eyes fixed on AOI
#    Maximum length of time for fixation (s)
MAXFIXTIME = 3      


# 5. Number of images displayed at each experiment
ntrials = 1  

# name used for log files, make this something unique (for example dependent on timestamp)
l = [time.strftime("%y%m%d", time.localtime()), 
     USER_NAME, 
     ANGLE, 
     'S'+str(int(SCREENDIST)),
     NOISE, 
     'D'+str(MAXTRIALTIME//1000), 
     'F'+str(MAXFIXTIME)]

LOGFILENAME = '_'.join([s for s in l if s!='']) 
print(LOGFILENAME)

# Experiment constants

In [ ]:
# some constants, which can later be stored in a constants.py file

# be used to determine where to store and retrieve data files
DIR = os.path.dirname('C:\\Users\\Chloe\\Documents\\Capstone\\')
# the DATADIR is the path to the directory where data files will be stored
DATADIR = os.path.join(DIR, 'data')
# the IMGDIR is the path to the directory that contains the image files
IMGDIR = os.path.join(DIR, 'images')

# path to logfile establishes where the gaze data will be written
LOGFILE = os.path.join(DATADIR, LOGFILENAME)
# how long the experimenter looks at the image after pressing 'space' (ms) -- transitioning time
FIXATIONTIME = 3000
# this is the type of eye tracker we will be using
TRACKERTYPE = 'opengaze'

# DISPLAY
# for the DISPTYPE, you can choose between 'pygame' and 'psychopy'; go for
# 'psychopy' if you need millisecond accurate display refresh timing, and go
# for 'pygame' if you experience trouble using PsychoPy
DISPTYPE = 'psychopy'
# the DISPSIZE is the monitor resolution, e.g. (1024,768)
DISPSIZE = (1920,1080) #(2880, 1800) #(1024,768)
# the SCREENSIZE is the physical screen size in centimeters, e.g. (39.9,29.9)
SCREENSIZE = (34.4, 19.3) #(33.0,21.0)

# set FULLSCREEN to True for fullscreen displaying, or to False for a windowed
# display, for experiments, set this to True, and for testing, set to False
FULLSCREEN = False
# BGC is for BackGroundColour, FGC for ForeGroundColour; both are RGB guns,
# which contain three values between 0 and 255, representing the intensity of
# Red, Green, and Blue respectively, e.g. (0,0,0) for black, (255,255,255) for
# white, or (255,0,0) for the brightest red
BGC = (0,0,0)
#BGC = (255,255,255)
FGC = (255,255,255)

TEXTSIZE = 12

Our gaze tracking equipment requires a display and screen on which the image data will be shown. Here, we initialize these objects. We then set up the experiment, including the calibration. We perform the experiment, and finally, close the relevant objects. I apologize for the long code block, but this needs to be run in one block.

In [ ]:
# read all image names
images = os.listdir(IMGDIR)
print(len(images))

# Experiment Display

In [ ]:
# initialize objects
disp = Display()
ip="127.0.0.1"
scr = Screen()

mouse = Mouse(mousebuttonlist=None, timeout=10000)
mouse.set_visible(visible=True)

# create keyboard object
kb = Keyboard(keylist=['space'], timeout=10000)
tracker = EyeTracker(disp, debug=True, logfile=LOGFILE)

# establish data connection to tracker
state=True
acknowledged, timeout = tracker.opengaze._send_message('SET', \
    'ENABLE_SEND_DATA', \
    values=[('STATE', int(state))], \
    wait_for_acknowledgement=True)
print(acknowledged)

# if you want to log any additional information, you will do it this way (saved to different file than gaze data)
log = Logfile()
log.write(["trialnr","image","imgtime"])

# set up experiment and perform calibration

# display instructions
scr.draw_text(text="Press any key to start the calibration.", fontsize=20)
disp.fill(scr)
disp.show()

# wait for a keypress
kb.get_key(keylist=None, timeout=None, flush=True)

# calibrate the eye tracker
tracker.calibrate()

# perform experiment

# loop through all trials
ntrials = 1  #len(images)
for trialnr in range(ntrials):
    
    # gerate random index
    i = randrange(len(images))
    
    # PREPARE TRIAL
    # draw the image
    scr.clear()
    scr.draw_image(os.path.join(IMGDIR,images[i]))
    
    # RUN TRIAL
    # start tracking
    tracker.start_recording()
    tracker.log("TRIALSTART %d" % trialnr)
    tracker.log("IMAGENAME %s" % images[i])
    tracker.status_msg("trial %d/%d" % (trialnr+1, ntrials))
    
    # present image
    disp.fill(scr)
    t0 = disp.show()
    tracker.log("image online at %d" % t0)
    
    start_time = time.time()
    while True:  
        # key is pressed or time exceeds limit
        if keyboard.is_pressed('space'):
            break
        elif (time.time() - start_time) > (MAXTRIALTIME / 1000):  
            break 
    
    timer.pause(FIXATIONTIME)
    
    # reset screen
    disp.fill()
    t1 = disp.show()
    tracker.log("image offline at %d" % t1)
    
    # stop recording
    tracker.log("TRIALEND %d" % trialnr)
    tracker.stop_recording()
    
    # TRIAL AFTERMATH
    # bookkeeping
    log.write([trialnr, images[i], t1-t0])
    
    # inter trial interval
    timer.pause(500)
    
# close experiment
# loading message
scr.clear()
scr.draw_text(text="Transferring the data file, please wait...", fontsize=20)
disp.fill(scr)
disp.show()

# neatly close connection to the tracker
# (this will close the data file, and copy it to the stimulus PC)
tracker.close()

# close the logfile
log.close()

# exit message
scr.clear()
scr.draw_text(text="This is the end of this experiment. Thank you for participating!\n\n(press any key to exit)", fontsize=20)
disp.fill(scr)
disp.show()

# wait for a keypress
kb.get_key(keylist=None, timeout=None, flush=True)

# close the Display
disp.close()